# Transportation Problem Optimization

This notebook solves a transportation problem using linear programming with the PuLP library.  
The goal is to minimize transportation cost while meeting supply and demand constraints.


In [ ]:
# Import PuLP library for optimization
import pulp


## Define Supply, Demand, and Costs

We have:
- Warehouses (suppliers) with limited supply.
- Stores (demand points) with required demand.
- Transportation cost per unit from each warehouse to each store.


In [ ]:
# Supply at each warehouse
supply = {'W1': 20, 'W2': 30, 'W3': 25}

# Demand at each store
demand = {'S1': 10, 'S2': 25, 'S3': 25, 'S4': 15}

# Transportation costs from each warehouse to each store
costs = {
    ('W1', 'S1'): 8, ('W1', 'S2'): 6, ('W1', 'S3'): 10, ('W1', 'S4'): 9,
    ('W2', 'S1'): 9, ('W2', 'S2'): 12, ('W2', 'S3'): 13, ('W2', 'S4'): 7,
    ('W3', 'S1'): 14, ('W3', 'S2'): 9, ('W3', 'S3'): 16, ('W3', 'S4'): 5
}


## Create the Linear Programming Problem

We define the problem as a minimization problem.


In [ ]:
# Define the LP problem
problem = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)


## Decision Variables

Define variables representing the number of units shipped from each warehouse to each store.


In [ ]:
# Decision variables: shipment quantities
routes = [(w, s) for w in supply for s in demand]
shipment_vars = pulp.LpVariable.dicts("Route", (supply.keys(), demand.keys()), lowBound=0, cat='Continuous')


## Objective Function

Minimize total transportation cost based on shipment quantities and per unit costs.


In [ ]:
# Objective function: minimize total cost
problem += pulp.lpSum([shipment_vars[w][s] * costs[(w, s)] for (w, s) in routes]), "Total_Transportation_Cost"


## Constraints

- Supply constraints: can't ship more than available supply at each warehouse.
- Demand constraints: must meet demand at each store.


In [ ]:
# Supply constraints
for w in supply:
    problem += pulp.lpSum([shipment_vars[w][s] for s in demand]) <= supply[w], f"Supply_Constraint_{w}"

# Demand constraints
for s in demand:
    problem += pulp.lpSum([shipment_vars[w][s] for w in supply]) >= demand[s], f"Demand_Constraint_{s}"


## Solve the Optimization Problem

Use PuLP's default solver to find the optimal solution.


In [ ]:
# Solve the problem
problem.solve()

# Print the status of the solution
print(f"Status: {pulp.LpStatus[problem.status]}\n")

print("Optimal Shipment Plan:")
for w in supply:
    for s in demand:
        quantity = shipment_vars[w][s].varValue
        if quantity and quantity > 0:
            print(f"  - Ship {quantity:.1f} units from Warehouse {w} to Store {s}")

print(f"\nTotal Transportation Cost = {pulp.value(problem.objective):.2f}")


## Insights and Conclusion

- The model optimizes transportation to minimize total cost.
- All store demands are met without exceeding warehouse supplies.
- The results provide actionable guidance for efficient logistics planning.
